# GEN AI Workflow Notebook ✨✨

ClassifAI allows users to create vector databases from text datasets, search those vector datasets in an ad-hoc retrieval manner, and deploy this pipeline as a restAPI service using FastAPI to perform AI assisted classification tasks.


An recent emergent AI field is Retrieval Augmented Generation (RAG) where text generation models that traditionally respond to a user 'prompt', first retrieve relevant infomration from a vector database via adhoc retrieval processes, and use those serach results as context to generate an answer for the original user prompt.

#### This notebook shows how we use RAG agents to perform classification on our VectorStore semantic search results!


## ClassifAIs existing retrieval setup

![Server_Image](files/servers.png)

#### The other modules of ClassifAI provide 3 core classes that work together to:

1. Vectorisers, to create embeddings,
2. Vectorstores, to create datgabsees of vectors and the ability to searc/query those database
3. RestAPI, to deploy a rest api service on a server to allow connections that search the created vector databases

In [ ]:
!uv pip install "classifai[gcp]"

## Setup

We need to first set up a traditional ClassifAI pipeline that can provide search/classification results for our generative model to use as context...


All of the following code is part of our standard ClassifAI setupm, and is a short demo of how you can create a semantic search classification system. <b>Check out our general_workflow_demo.ipynb notebook for a walkthrough of the content of these cells!</b>

In [ ]:
!gcloud auth application-default login

In [ ]:
from classifai.indexers import VectorStore
from classifai.vectorisers import HuggingFaceVectoriser

# Our embedding model is pulled down from HuggingFace, or used straight away if previously downloaded
# This also works with many different huggingface models!
vectoriser = HuggingFaceVectoriser(model_name="sentence-transformers/all-MiniLM-L6-v2")


my_vector_store = VectorStore(
    file_name="./data/fake_soc_dataset.csv",  # demo csv file from the classifai package repo! (try some of our other DEMO/data datasets)
    data_type="csv",
    vectoriser=vectoriser,
    agent=None,
    overwrite=True,
)

## We've set up our semantic search classification system

The following cell runs the 'search' method of the vectorstore, which will take a query for our created vectorstore and return the top k (n_results) most similar samples stored in the vectorstore. This is an example of our exisiting retrieval capabilities with the ClassifAI package. 

This retrieved set can then be used to make a final classification decision, by some method such as:
- automatically choosing the top ranked item, 
- a human in the loop physically looking at the retrieved candidates making the decision,
- by using a generative AI agent to assess the candidate lists and making a final decison... more on that later

In [ ]:
from classifai.indexers.dataclasses import VectorStoreSearchInput

input_object = VectorStoreSearchInput(
    {"id": [1, 2], "query": ["dairy famer that sells milk", "happy construction worker"]}
)

bb = my_vector_store.search(input_object, n_results=5)
bb

## Creating an AI Agent

With our semantic search classification pipeline set up, we can send the top K results (seen above) to a Genereative AI model and ask that LLM to make a final decision on which result is the correct result for the user's original input query. Passing semantic search results to a generatieve model for some task is often referred to as 'Retrieval Augmented Generation'.


![Rag_Image](files/agent.png) 


Our `GcpAgent` class agent has a transform() method which accepts a `VectorStoreSearchOutput` object. Passing this `VectorStoreSearchOutput` object to the transform() method will return another `VectorStoreSearchOutput` object, which will modify the results in some way. The classificaion GcpAgent, reduces the semantic search results down to a single result row for each query in the VectorStoreSearchOutput results.




To instantiate the GcpAgent, the constructor takes:

- project_id - of the google project associated with the Google Gemini embedding models
- location - corresponding to the Gcloud Project
- model_name - the specific LLM model to use, that is avaiable on Gcloud
- task_type - indicates the kind of work you want the LLM to do on the ClassifAI results

In [ ]:
from classifai.agents import GcpAgent

my_agent = GcpAgent(
    project_id="platforms-sandbox", location="europe-west2", model_name="gemini-2.5-flash", task_type="classification"
)

We can pass some VectorStore search results directly to the agents transform() method

In [ ]:
semantic_search_results = my_vector_store.search(input_object, n_results=5)

print(type(semantic_search_results))
semantic_search_results

#### To use the If the classification agent can make a decision, it will return a single row for the corresponding query_id. Otherwise it will return the original results


In [ ]:
my_agent.transform(semantic_search_results)

#### To use the agent in our ClassifAI pipeline, we can attach it to the running VectorStore

In [ ]:
my_vector_store.agent = my_agent

#### We can then run vectorstore.search and it will automatically call the agent to process the results!

In [ ]:
my_vector_store.search(input_object, n_results=5)

## Thats it!

You can see in the above cell that my_agent.transform() takes and returns a VectorStoreSearchOutput object and therefore integrates with the VectorStore search method.

Check out the general_worfklow notebook to see how VectorStores can be deployed, and you'll find that the Agent can be deployed as an integrated part of the VectorStore.